# <center>Telecom Churn : Prediction</center> 

## Problem Statement<br>
Analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.<br>
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. The telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.
## Goal
- Analyse customer data with **Prepaid** pland in **India and Southest Asia**
- Identify High-Value customer based on there uses
    - Take first 2 month average recharge amount
    - Identify customer with more than **70<sup>th</sup> Percentile** average recharge amount 
- Here we have 4 month data (Jun, July, August, September)
    - Tag churner in the last month (September) using fourth month data
    - Based on usage of fourth month such as incoming and outgoing call and internet use
    - After prediction remove all the attributes corresponding to the churn phase


## Load Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime as dt
import warnings
%matplotlib inline

In [ ]:
from IPython.display import display, Markdown
def customprint(text):
    display(Markdown(text))

In [ ]:
pd.set_option('display.max.columns', 250)
#warnings.filterwarnings('ignore')

## Load Data

In [ ]:
custDf = pd.read_csv("telecom_churn_data.csv")

In [ ]:
custDf.shape

In [ ]:
custDf.head()

## Data Cleaning and Manipulation

In [ ]:
#changing few column names
custDf.rename(columns={'jun_vbc_3g':'vbc_3g_6', 'jul_vbc_3g':'vbc_3g_7', 'aug_vbc_3g':'vbc_3g_8', 'sep_vbc_3g':'vbc_3g_9'}, inplace=True)

Indentify **High-Value** customer

In [ ]:
#Check for different recharge amount and recharge count
custDf[[c for c in custDf.columns if ((c.endswith('_6')) 
                                      & (('rch' in c) | 
                                         ('rech' in c) | 
                                         ('sachet' in c) | 
                                         ('monthly' in c) | 
                                         ('night' in c) | 
                                         ('VBC' in c)))]].tail(5)

In [ ]:
custDf[['total_rech_num_6','total_rech_data_6', 'count_rech_2g_6', 'count_rech_3g_6', 'sachet_2g_6', 'sachet_3g_6','monthly_2g_6','sachet_2g_6','monthly_3g_6']][~np.isnan(custDf['total_rech_data_6'])].head()

In [ ]:
custDf[['total_rech_amt_6','max_rech_amt_6', 'max_rech_data_6', 'av_rech_amt_data_6']][~np.isnan(custDf['total_rech_data_6'])].head()

In [ ]:
print("Missing value : Average Recharge Amount for data (June) : ", custDf[np.isnan(custDf['av_rech_amt_data_6']) & custDf['total_rech_data_6'] > 0].size)
print("Missing value : Average Recharge Amount for data (July) : ", custDf[np.isnan(custDf['av_rech_amt_data_7']) & custDf['total_rech_data_7'] > 0].size)
print("Missing value : Average Recharge Amount for data (August) : ", custDf[np.isnan(custDf['av_rech_amt_data_8']) & custDf['total_rech_data_8'] > 0].size)

In [ ]:
#Calculate Total recharge amount spned on data for specific month
custDf['total_rech_amt_data_6'] = custDf['av_rech_amt_data_6'] * custDf['total_rech_data_6']
custDf['total_rech_amt_data_7'] = custDf['av_rech_amt_data_7'] * custDf['total_rech_data_7']
custDf['total_rech_amt_data_8'] = custDf['av_rech_amt_data_8'] * custDf['total_rech_data_8']

In [ ]:
custDf['total_rech_6'] = custDf[['total_rech_amt_data_6', 'total_rech_amt_6']].sum(axis=1)
custDf['total_rech_7'] = custDf[['total_rech_amt_data_7', 'total_rech_amt_7']].sum(axis=1)
custDf['total_rech_8'] = custDf[['total_rech_amt_data_8', 'total_rech_amt_8']].sum(axis=1)
custDf['av_rech_6_7'] = custDf[['total_rech_6','total_rech_7']].mean(axis=1)

In [ ]:
#70th percentile of two month average recharge  
rechAmt = custDf['av_rech_6_7'].quantile(0.7)
rechAmt

In [ ]:
#High value customer with average recharge amount greater than 70 percentile
hvCust = custDf[custDf['av_rech_6_7'] >= rechAmt]

In [ ]:
hvCust.shape

In [ ]:
#Will drop 'av_rech_6_7' columns which might create confusion in EDA process
hvCust.drop(labels=['av_rech_6_7'], axis=1, inplace=True)

Tag churner based on Incomeing & Outgoing call as well as Internet usages in september month.

In [ ]:
hvCust['churn'] = hvCust.apply(lambda x : 1 if ((x['total_ic_mou_9'] == 0) & (x['total_og_mou_9'] == 0) & (x['vol_2g_mb_9'] == 0) & (x['vol_3g_mb_9'] == 0)) else 0, axis=1)

In [ ]:
hvCust[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9', 'churn']].head(10)

In [ ]:
#Drop attribute from churn phase
colsToDel = [c for c in hvCust.columns if "_9" in c]
hvCust.drop(labels=colsToDel, inplace=True, axis=1)
hvCust.shape

In [ ]:
# as name suggeted "Last Date of Month" column contain unique value (Last day of that month)
#So we can drop these 3 columns
hvCust.drop(labels=['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8'], inplace=True, axis=1)

In [ ]:
hvCust.info()

In [ ]:
#Fetch day from last recharge date of specific month
hvCust['date_of_last_rech_6'] = pd.DatetimeIndex(hvCust['date_of_last_rech_6']).day
hvCust['date_of_last_rech_7'] = pd.DatetimeIndex(hvCust['date_of_last_rech_7']).day
hvCust['date_of_last_rech_8'] = pd.DatetimeIndex(hvCust['date_of_last_rech_8']).day
hvCust['date_of_last_rech_data_6'] = pd.DatetimeIndex(hvCust['date_of_last_rech_data_6']).day
hvCust['date_of_last_rech_data_7'] = pd.DatetimeIndex(hvCust['date_of_last_rech_data_7']).day
hvCust['date_of_last_rech_data_8'] = pd.DatetimeIndex(hvCust['date_of_last_rech_data_8']).day

In [ ]:
hvCust.info()

In [ ]:
missingDf = pd.DataFrame(data=hvCust.isnull().sum() / hvCust.index.size * 100, columns=['MissingPercent'])
missingDf =  missingDf[missingDf['MissingPercent'] > 0]
missingDf.reset_index(inplace=True)
missingDf.columns = ['Feature', 'MissingPercent']
missingDf[['Month', 'Feature']] = missingDf['Feature'].apply(lambda x : pd.Series([6, x.replace('_6', '')] if x.endswith('_6') else ([7, x.replace('_7', '')] if x.endswith('_7') else ([8, x.replace('_8', '')] if x.endswith('_8') else [None, x]))))

In [ ]:
pvtDf =  missingDf[~np.isnan(missingDf['MissingPercent'])].pivot_table(index=['Feature'], columns=['Month'])
pvtDf['MissingPercent'].sort_values(by=[6.0], ascending=False)

In [ ]:
#hvCust[hvCust['total_rech_amt_6'] == 0][[c for c in hvCust.columns if '_6' in c]]
hvCust[np.isnan(hvCust['total_rech_data_6']) | 
      np.isnan(hvCust['total_rech_amt_data_6']) |
      np.isnan(hvCust['night_pck_user_6']) |
      np.isnan(hvCust['max_rech_data_6']) |
      np.isnan(hvCust['fb_user_6']) |
      np.isnan(hvCust['count_rech_3g_6']) |
      np.isnan(hvCust['count_rech_2g_6']) |
      np.isnan(hvCust['av_rech_amt_data_6']) |
      np.isnan(hvCust['arpu_3g_6']) |
      np.isnan(hvCust['arpu_2g_6'])][[c for c in hvCust.columns if '_6' in c]].head(10)

Apart from **date_of_last_rech** column all other column have some common pattern.<BR>
All feature variable related to Internet Service have **44% missing data in Jun, 43% in July and 47% in August ** month.<BR>
Same as internet service, calling service related features also have common trend.<BR>

This trend shows there are few customers only use calling service and not using any internet service, hence approx. 44% missing data for all columns of internet service related feature.<BR>
We can impute zero in such columns except last recharge date data column.

In [ ]:
hvCust.drop(labels=['date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_8'], axis=1, inplace=True)

In [ ]:
cols = pvtDf[pvtDf['MissingPercent'][6] > 40].index
cols = [c + "_" + month for c in cols for month in ['6','7','8']]
cols.remove('date_of_last_rech_data_6')
cols.remove('date_of_last_rech_data_7')
cols.remove('date_of_last_rech_data_8')

In [ ]:
hvCust[cols] = hvCust[cols].fillna(value=0)

In [ ]:
#Drop rows of other missing data
hvCust.dropna(axis=0, inplace=True)

In [ ]:
missingDf = pd.DataFrame(data=hvCust.isnull().sum() / hvCust.index.size * 100, columns=['MissingPercent'])
missingDf =  missingDf[missingDf['MissingPercent'] > 0]
missingDf.reset_index(inplace=True)
missingDf.columns = ['Feature', 'MissingPercent']

In [ ]:
missingDf.sort_values(by='MissingPercent')

In [ ]:
print(hvCust.shape)
print(hvCust['churn'].value_counts())

As per defination **Good Phase** (i.e. June & July), customer is happy with service provider and in **Action Phase** (i.e. August) it shows show different behaviour.<BR>
So will combine June & July data points for analsysi and futher use.

In [ ]:
#Will exclude few variable such as categorical (0,1) and related to date/day
colToExclude = ['date_of_last_reach_6','date_of_last_reach_7','fb_user_6','fb_user_7','night_pck_user_6','night_pck_user_7']
for col in hvCust.columns:
    if (col.endswith('_6')) & (col not in colToExclude):
        hvCust[col + '_7'] = hvCust[[col, col.strip('_6') + '_7']].mean(axis=1)
        hvCust.drop(labels=[col, col.strip('_6') + '_7'], inplace=True, axis=1)

In [ ]:
#Duplicate data check
hvCust[hvCust['mobile_number'].duplicated()]

In [ ]:
hvCust.describe()

Few feature variables contain only **0** value, we can drop such variables.<BR>
Also **circle_id** feature have common value **109**, we can drop this feature also.

In [ ]:
s = hvCust.index.size
tempDf = pd.DataFrame([[c, hvCust[hvCust[c] == 0].index.size / s] for c in hvCust.columns if (hvCust[hvCust[c] == 0].index.size / s > 0.7)])
tempDf.columns = ['Feature', '% of 0']
tempDf.sort_values(by=['% of 0'], ascending = False)

In [ ]:
#Will drop features having more than 98% zero values
colToDrop = list(tempDf[tempDf['% of 0'] > 0.98]['Feature'])
colToDrop.append('circle_id')
hvCust.drop(labels=colToDrop, inplace=True, axis=1)

## Visualising data

In [ ]:
#Correlation of Good Phase Data
col = [c for c in hvCust.columns if c.endswith('_6_7') ] + ['churn']
corrData_6_7 = hvCust[col].corr()
plt.figure(figsize=(15,12))
mask = np.zeros_like(corrData_6_7, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corrData_6_7, cmap='RdBu', mask=mask, center=0, linewidths= 0.1)
plt.show()

In [ ]:
#Correlation of Action Phase Data
col = [c for c in hvCust.columns if c.endswith('_8') ] + ['churn']
corrData_8 = hvCust[col].corr()
plt.figure(figsize=(15,12))
mask = np.zeros_like(corrData_8, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corrData_8, cmap='RdBu', mask=mask, center=0, linewidths= 0.1)
plt.show()

Both heatmap shows similar trend, will drop few highly correlated column.

In [ ]:
for c in corrData_6_7.columns:
    corrCol = corrData_6_7[(corrData_6_7[c] >= 0.8) & (corrData_6_7[c] < 1)].index
    if (corrCol.size > 0):
        print("'{0}' correlated with '{1}' : {2}".format(c, corrCol[0], corrData_6_7.loc[c, corrCol[0]]))
print()        
print('-----------------------------------------------------------------')
print()
for c in corrData_8.columns:
    corrCol = corrData_8[(corrData_8[c] >= 0.8) & (corrData_8[c] < 1)].index
    if (corrCol.size > 0):
        print("'{0}' correlated with '{1}' : {2}".format(c, corrCol[0], corrData_8.loc[c, corrCol[0]]))

In [ ]:
#Let's drop highly correlated Columns
colToDrop = ['total_rech_amt_6_7','total_rech_amt_8','std_og_t2t_mou_6_7','std_og_t2t_mou_8','std_og_t2m_mou_6_7','std_og_t2m_mou_8','std_og_mou_6_7','std_og_mou_8','loc_ic_mou_6_7','loc_ic_mou_8','std_ic_t2m_mou_6_7','std_ic_t2m_mou_8','total_rech_data_6_7','total_rech_data_8','sachet_3g_6_7','sachet_3g_8','sachet_2g_6_7','sachet_2g_8']
hvCust.drop(labels=colToDrop, inplace=True, axis=1)

In [ ]:
hvCust.info()

In [ ]:
cols = ['fb_user_6','fb_user_7','fb_user_8','churn']
n_cols = 4
n_rows = math.ceil(len(cols) / n_cols)
plt.figure(figsize=(12,4))
for i,c in enumerate(sorted(cols)):
    plt.subplot(n_rows, n_cols, i+1)
    ax = sns.countplot(x=hvCust[c])
    ax.set_xlabel('')
    ax.set_ylabel('')

    plt.title(c.capitalize(), y=1, fontsize=15)
    
plt.show()

In [ ]:
cols = hvCust.columns[~hvCust.columns.isin(['mobile_number','fb_user_6','fb_user_7','fb_user_8','churn'])]
n_cols = 3
n_rows = math.ceil(len(cols) / n_cols)
plt.figure(figsize=(15,100))
for i,c in enumerate(sorted(cols)):
    plt.subplot(n_rows, n_cols, i+1)
    ax = sns.distplot(hvCust[c])
    ax.set_xlabel('')
    
    plt.title(c.capitalize(), y=0.85, fontsize=15)
    
plt.show()

All feature variables are either skewed to right or left, not a single variable showing normal distribution.<BR>
We are analyzing high value customers which is nothing but outliers, so we can expect such trend in all other variable. Hence instead of removing outlier data will start building model.

## Model Building

In [ ]:
#data Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, IncrementalPCA

#Util
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold

#Model Algo
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#Model Evluation
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, precision_recall_curve, recall_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
#Split Train & Test Data
Y = hvCust['churn']
X = hvCust.drop(labels=['churn','mobile_number'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state = 100, stratify = Y)
print("Train Set :", X_train.shape, Y_train.shape)
print("Test Set :", X_test.shape, Y_test.shape)

In [ ]:
#Scale data using Standard Scaler
sc = StandardScaler()
X_train[X_train.columns] = sc.fit_transform(X_train[X_train.columns])
X_test[X_test.columns] = sc.transform(X_test[X_test.columns])

In [ ]:
#Instead of building model on all feature will fetch important feature using RFE
rfe = RFE(LogisticRegression(), n_features_to_select=60)
rfe.fit(X_train, Y_train)

In [ ]:
rfeDF = pd.DataFrame(data=np.array([X_train.columns, rfe.support_, rfe.ranking_]).T, columns=['Feature', 'Support', 'Ranking'])
rfeDF[rfeDF['Support']]

In [ ]:
pca = PCA(svd_solver='randomized', random_state=100)
pca.fit(X_train[rfeDF[rfeDF['Support']]['Feature']])
#pca.fit(X_train)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.yticks(np.arange(0,1.1,0.1))
plt.grid(linestyle='-', linewidth = 0.5)
plt.xlabel('Principle Components')
plt.ylabel('Variance Explained')
plt.show()

In [ ]:
np.cumsum(pca.explained_variance_ratio_)[np.arange(29,60,5)]

In [ ]:
np.cumsum(pca.explained_variance_ratio_)#[np.arange(30,60,5)]

**91%** variance explained by **30** PC<BR>
**94%** variance explained by **35** PC<BR>
**96%** variance explained by **40** PC<BR>
**98%** variance explained by **45** PC<BR>
**99%** variance explained by **50** PC<BR>
**100%** variance explained by **55** PC<BR>
So, we can choose **40 or 45** PC for model building which explained **96% - 98%** variance.

In [ ]:
pca = IncrementalPCA(n_components=40)
pca_train = pca.fit_transform(X_train[rfeDF[rfeDF['Support']]['Feature']])
pca_test = pca.transform(X_test[rfeDF[rfeDF['Support']]['Feature']])

In [ ]:
folds = StratifiedKFold(n_splits = 5, random_state = 100, shuffle = True)

In [ ]:
lr = LogisticRegression()
param = {'C' : [1,10,100,1000]}
model_cv = GridSearchCV(lr, param_grid=param, scoring='recall', cv=folds, n_jobs=-1, verbose=1, return_train_score=True)
model_cv.fit(pca_train, Y_train)

In [ ]:
lrScore = pd.DataFrame(model_cv.cv_results_)
lrScore[['param_C','mean_test_score','mean_train_score']]

In [ ]:
asdas

In [ ]:
Y_train.value_counts()

In [ ]:
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 100)
param = {'C' : [0.001,0.1,1]}
svc = SVC(kernel='linear')
model_scv_li = GridSearchCV(svc,
                             param_grid=param, 
                             scoring='precision', 
                             cv=folds, 
                             verbose=1,
                             n_jobs = -1,
                             return_train_score=True)
model_scv_li.fit(pca_train, Y_train)

In [ ]:
model_scv_li_Score = pd.DataFrame(model_scv_li.cv_results_)
model_scv_li_Score[['param_C','mean_test_score','mean_train_score']]

In [ ]:
svc = SVC(kernel='linear')
param = {'C' : [0.01, 0.1, 1, 10]}
#param = {'C' : [0.1]}
model_svc_lr = GridSearchCV(svc, param_grid=param, cv=2, scoring='accuracy', n_jobs=-1, verbose=1, return_train_score=True)
model_svc_lr.fit(pca_train, Y_train)

In [ ]:
scvlrScore = pd.DataFrame(model_svc_lr.cv_results_)
scvlrScore[['param_C','mean_test_score','mean_train_score']]

param = {'C' : [0.01, 0.1, 1, 10]}
svc = SVC(kernel='linear')
model_scv_li = GridSearchCV(svc,
                             param_grid=param, 
                             scoring='accuracy', 
                             cv=folds, 
                             verbose=1,
                             n_jobs = -1,
                             return_train_score=True)
model_scv_li.fit(pca_train, Y_train)

In [ ]:
svc = SVC(kernel='poly')
param = {'C' : [0.01, 0.1, 1, 10],
        'gamma': [0.01, 0.1, 1]}
model_svc_poly = GridSearchCV(svc, param_grid=param, cv=folds, scoring='recall', n_jobs=-1, return_train_score=True)
model_svc_poly.fit(pca_train, Y_train)

scvpolyScore = pd.DataFrame(model_svc_poly.cv_results_)
scvpolyScore[['param_C','mean_test_score','mean_train_score']]

In [ ]:
svc = SVC(kernel='rbf')
param = {'C' : [0.01, 0.1, 1, 10],
        'gamma': [0.01, 0.1, 1]}
model_svc_rbf = GridSearchCV(svc, param_grid=param, cv=folds, scoring='recall', n_jobs=-1, return_train_score=True)
model_svc_rbf.fit(pca_train, Y_train)

scvrbfScore = pd.DataFrame(model_svc_rbf.cv_results_)
scvrbfScore[['param_C','mean_test_score','mean_train_score']]